# LOTR (Lord of the Retreivers)

Lord of the Retrievers, also known as MergerRetriever, takes a **list of retrievers** as input and merges the results of their get_relevant_documents() methods into a single list. The merged results will be a list of documents that are relevant to the query and that have been ranked by the different retrievers.

The **MergerRetriever** class can be used to improve the accuracy of document retrieval in a number of ways.

* First, it can combine the results of multiple retrievers, which can help to reduce the risk of bias in the results.
* Second, it can rank the results of the different retrievers, which can help to ensure that the most relevant documents are returned first.

## Installing neccessary libraries

In [1]:
%%capture
!pip install -U langchain langchain-openai huggingface_hub langchain-community
!pip install lancedb pypdf sentence-transformers openai
!pip install tiktoken
!pip install lancedb
!pip install sentence-transformers
!pip install openai
!pip install pypdf

In [2]:
import os

os.environ["OPENAI_API_KEY"] = "sk-proj-...."

## Import the libraries

In [3]:
from langchain.embeddings import (
    HuggingFaceEmbeddings,
    OpenAIEmbeddings,
    HuggingFaceBgeEmbeddings,
)
import os
from langchain.document_transformers import (
    EmbeddingsClusteringFilter,
    EmbeddingsRedundantFilter,
)
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import DocumentCompressorPipeline
from langchain.retrievers.merger_retriever import MergerRetriever
from langchain.vectorstores import LanceDB
import lancedb
from langchain.schema import Document
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.document_loaders import PyPDFLoader
from langchain.embeddings import SentenceTransformerEmbeddings
from re import search
from langchain.document_transformers import LongContextReorder
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI

### Download the data (you can change it to any pdf you like)

In [ ]:
!wget https://raw.githubusercontent.com/lancedb/vectordb-recipes/main/examples/Advance_RAG_LOTR/Ayurvedic-Home-Remedies-English.pdf

## Defining the Embeddings model

In [10]:
medical_health_embedding = SentenceTransformerEmbeddings(
    model_name="NeuML/pubmedbert-base-embeddings"
)

all_mini = HuggingFaceEmbeddings(model_name="google/t5-efficient-small-dm768")

filter_embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")  ## second embedding model

config.json:   0%|          | 0.00/635 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/363M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.98k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


## Load the data

In [5]:
# load single pdfz
loader = PyPDFLoader("Ayurvedic-Home-Remedies-English.pdf")
pages = loader.load_and_split()

## Define the text splitter

In [6]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
text = text_splitter.split_documents(pages)
# text_esops = text_splitter.split_documents(documnet_esops)

### First Embedding

In [ ]:
# embedding model 1
# Initialize LanceDB retriever
db_all = LanceDB.from_documents(text, all_mini)

### Second Embedding

In [13]:
# embedding model 2
# Initialize LanceDB retriever
db_multiqa = LanceDB.from_documents(text, medical_health_embedding)

### Merge all the retrievers
This will hold the outputs from both the retrievers and can be used as any other retriever on different types of chains.


In [14]:
# med retriever
retriever_med = db_all.as_retriever(
    search_type="similarity", search_kwargs={"k": 5}
)

# bge embeddings
retriever_bge = db_multiqa.as_retriever(
    search_type="similarity", search_kwargs={"k": 5}
)

In [16]:
# merger retriever
lotr = MergerRetriever(retrievers=[retriever_med, retriever_bge])

for chunks in lotr.get_relevant_documents("What is use of tulsi ?"):
    print(chunks.page_content)

<ipython-input-16-f2ef47f3bf89>:5: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use invoke instead.
  for chunks in lotr.get_relevant_documents("What is use of tulsi ?"):


In general too spicy, salty, chilly, sour, preserved 
items fried food, heavy, indigestible, too cold & hot, 
stale food and the food that do not suit the health 
should be avoided.
Irregular food habit, sleep and lack of physical 
exercise is main cause for may diseases.
Too much tea, coffee, should be avoided. Tobacco, 
alcohol and drugs should not be taken.
Mental stress should be tackled by recreation such 
as meditation, prayer, sports, exercises, yoga & 
other activities of individual's choice.
TULSI
(Ocimum sanctum,  Tulasi)
Cough/ Cold 
Fever
Skin allergy
Indigestion/ 
Loss of appetite
Greying of hair 
W ound/ulcer 
Ear pain5-10 ml. juice twice or thrice a day 
with honey.
30 ml. decoction from handful of 
leaves & 5 gm. Dhania  thrice a 
day.
5-10 ml. juice twice or thrice daily. 
5-10 ml. juice twice or thrice daily.
Coconut oil processed with Tulsi  
juice for regular use.
Juice mixed with honey & Haldi  
powder for application.
2-3 luke warm drops 2 times daily 
(D o n o t 

### Remove **redundant results** from the merged retrievers
EmbeddingsRedundantFilter drops redundant documents by comparing their embeddings

In [17]:
# Remove redundant results from the merged retrievers EmbeddingsRedundantFilter drops redundant documents by comparing their embeddings
filter = EmbeddingsRedundantFilter(embeddings=all_mini)

reordering = LongContextReorder()

pipeline = DocumentCompressorPipeline(transformers=[filter, reordering])
compression_retriever_reordered = ContextualCompressionRetriever(
    base_compressor=pipeline,
    base_retriever=lotr,
    search_kwargs={"k": 5},
)

docs = compression_retriever_reordered.get_relevant_documents("What is use of tulsi ?")
print(len(docs))

print(docs[0].page_content)

9
TULSI
(Ocimum sanctum,  Tulasi)
Cough/ Cold 
Fever
Skin allergy
Indigestion/ 
Loss of appetite
Greying of hair 
W ound/ulcer 
Ear pain5-10 ml. juice twice or thrice a day 
with honey.
30 ml. decoction from handful of 
leaves & 5 gm. Dhania  thrice a 
day.
5-10 ml. juice twice or thrice daily. 
5-10 ml. juice twice or thrice daily.
Coconut oil processed with Tulsi  
juice for regular use.
Juice mixed with honey & Haldi  
powder for application.
2-3 luke warm drops 2 times daily 
(D o n o t use w hen th ere is 
discharge).
25


### Chat with the data

In [18]:
llm = ChatOpenAI(openai_api_key=os.environ["OPENAI_API_KEY"])

qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=compression_retriever_reordered,
    return_source_documents=True,
)

In [19]:
query = "What is use of tulsi?"
results = qa(query)
print(results["result"])

print(results["source_documents"])

<ipython-input-19-a3e1f0ab1727>:2: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  results = qa(query)


Tulsi, also known as Ocimum sanctum, has various uses in traditional medicine. It is used for cough, cold, fever, skin allergies, indigestion, loss of appetite, greying of hair, wounds, ulcers, and ear pain. The juice of Tulsi can be taken with honey for cough, cold, and fever. It can also be used topically for skin allergies and hair care. Additionally, Tulsi can be processed with coconut oil for regular use and can be made into a decoction for various ailments.
[_DocumentWithState(metadata={'page': 31, 'source': 'Ayurvedic-Home-Remedies-English.pdf'}, page_content='TULSI\n(Ocimum sanctum,  Tulasi)\nCough/ Cold \nFever\nSkin allergy\nIndigestion/ \nLoss of appetite\nGreying of hair \nW ound/ulcer \nEar pain5-10 ml. juice twice or thrice a day \nwith honey.\n30 ml. decoction from handful of \nleaves & 5 gm. Dhania  thrice a \nday.\n5-10 ml. juice twice or thrice daily. \n5-10 ml. juice twice or thrice daily.\nCoconut oil processed with Tulsi  \njuice for regular use.\nJuice mixed with 

In [20]:
query = "having high fiver & cough what should i take as home remadies?"
results = qa(query)
print(results["result"])

# print(results["source_documents"])

For a high fever and cough, you can try the following home remedies from the context provided:

1. **Tayphal (Myristica fragrans)**:
   - For cough, you can try rubbing a pinch of Tayphal powder on a clean surface with milk or water and give it to the person 3-4 times a day.
   
2. **Elaichi (Elettaria cardamomum)**:
   - For cough, you can take 20 ml of a decoction prepared from 5 gm Dhania, 1 gm Methi seeds, and a little turmeric powder 2-3 times a day.
   - You can also try taking a little Elaichi powder with a teaspoon of honey 3-4 times a day.
   
3. **Madhu (Honey)**:
   - You can try taking honey with a pinch of pepper powder three to four times a day.
   - Additionally, you can mix honey with ghee and apply it.
   
Please note that these remedies are based on traditional Ayurvedic practices. If the symptoms persist or worsen, it's important to consult a healthcare professional for proper diagnosis and treatment.
